In [22]:
import os
import sionna
import tensorflow as tf

tf.random.set_seed(1) # Set global random seed for reproducibility

In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [24]:
os.chdir("/home/ub2204/桌面/g2c/SISO_simulate_v2")

In [25]:
# Load integrated scene
scene = load_scene('NSYSU/xml/NSYSU2.xml')
scene.frequency = 3.5e9
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [26]:
obj = scene.get("map_4_osm_buildings-wall") 
obj.radio_material = "itu_concrete"

In [27]:
# Select an example object from the scene
so = scene.get("itu_concrete")

# Print name of assigned radio material for different frequenies
for f in [3.5e9, 2.14e9]: # Print for differrent frequencies
    scene.frequency = f
    print(f"\nRadioMaterial: {so.name} @ {scene.frequency/1e9:.2f}GHz")
    print("Conductivity:", so.conductivity.numpy())
    print("Relative permittivity:", so.relative_permittivity.numpy())
    print("Complex relative permittivity:", so.complex_relative_permittivity.numpy())
    print("Relative permeability:", so.relative_permeability.numpy())
    print("Scattering coefficient:", so.scattering_coefficient.numpy())
    print("XPD coefficient:", so.xpd_coefficient.numpy())
so.scattering_coefficient = tf.Variable(0.1, dtype=tf.float32)


RadioMaterial: itu_concrete @ 3.50GHz
Conductivity: 0.123086944
Relative permittivity: 5.24
Complex relative permittivity: (5.24-0.63214296j)
Relative permeability: 1.0
Scattering coefficient: 0.0
XPD coefficient: 0.0

RadioMaterial: itu_concrete @ 2.14GHz
Conductivity: 0.0837706
Relative permittivity: 5.24
Complex relative permittivity: (5.24-0.7036379j)
Relative permeability: 1.0
Scattering coefficient: 0.0
XPD coefficient: 0.0


In [28]:
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[149.5, -97.1, 26],
                 orientation=[0, 0, 0])

Tx_pos = np.array([149.5, -97.1, 26]).reshape(-1, 3)

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=[149.5, -97.1, 1.5],
              orientation=[0,0,0])

# Add receiver instance to scene
scene.add(rx)

scene.synthetic_array = True

tx.look_at(rx) # Transmitter points towards receiver

scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…

In [29]:
cm = scene.coverage_map(max_depth=8, 
                        los=True, 
                        reflection=True, 
                        diffraction=True, 
                        check_scene=False) 

In [30]:
paths1 = scene.compute_paths(max_depth=8,
                            method='fibonacci',
                            num_samples=1e5,
                            los = True,
                            reflection=True,
                            diffraction=True,
                            check_scene=True,
                            scattering=False,
                            edge_diffraction=True,
                            scat_keep_prob=0.01)
    
ray_num = len(paths1.types.numpy()[0])
print(f"ray number : {ray_num}")

ray number : 0


In [23]:
"""min_gain_db = -160 # in dB; ignore any position with less than -130 dB path gain
# max_gain_db = 0 # in dB; ignore strong paths

# sample points in a 5-400m radius around the receiver
min_dist = 0 # in m
max_dist = 1000 # in m

batch_size = 13000

#sample batch_size random user positions from coverage map
ue_pos = cm.sample_positions(batch_size=batch_size,
                             min_gain_db=min_gain_db,
                             # max_gain_db=max_gain_db,
                             min_dist=min_dist,
                             max_dist=max_dist)

# present time only YYYYMMDD_HH
np.save(f"NSYSU_13k/ue_pos_13k.npy", ue_pos)"""

In [40]:
ue_pos = np.load("/home/ub2204/桌面/g2c/ue_pos_grid.npy")
print(ue_pos.shape)
batch_size = ue_pos.shape[0]

(16384, 3)


In [41]:
def set_rx_and_compute(scene, ue_pos, idx):
    rx = Receiver(name=f"rx-{idx}",
                  position=ue_pos[idx])
    scene.add(rx)
    
    paths = scene.compute_paths(max_depth=8,
                            method='fibonacci',
                            num_samples=1e5,
                            los = True,
                            reflection=True,
                            diffraction=True,
                            check_scene=True,
                            scattering=False,
                            edge_diffraction=True,
                            scat_keep_prob=0.01)
    
    ray_num = len(paths.types.numpy()[0])
    print(f"ray number : {ray_num}")

    return paths, ray_num

In [42]:
def get_ray_properties(paths, ray_num):
    ray_properties = []
    for r in range(ray_num):
        path_idx = r
        path_coefficients = paths.a[0,0,0,0,0,path_idx, 0].numpy()
        if path_coefficients == 0: # Skip paths with zero gain
            continue

        path_gain = np.abs(path_coefficients)
        path_phase = np.angle(path_coefficients)
        path_delay = paths.tau[0,0,0,path_idx].numpy()
        path_AOD_ver = paths.theta_t[0,0,0,path_idx]
        path_AOD_hor = paths.phi_t[0,0,0,path_idx]
        path_AOA_ver = paths.theta_r[0,0,0,path_idx]
        path_AOA_hor = paths.phi_r[0,0,0,path_idx]

        ray_properties.append([path_gain, path_phase, path_delay, path_AOD_hor, path_AOD_ver, path_AOA_hor, path_AOA_ver])
    
    return ray_properties


In [43]:
def pad_to_297(ray_properties):
    while len(ray_properties) < 297:
        ray_properties.append([0, 0, 0, 0, 0, 0, 0])
    
    return ray_properties

In [28]:
scene.remove("rx")

# Configure antenna array for all receivers (=UEs)
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1, # Each receiver is equipped with 4 tx antennas (uplink)
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso", # UE orientation is random
                             polarization="V")


zero_gain_ue = []
building_pixel = []

for ue in range(batch_size):
    rx = Receiver(name=f"rx-{ue}",
                  position=ue_pos[ue], # Random position sampled from coverage map
                  )
    scene.add(rx)
    
    paths = scene.compute_paths(max_depth=8,
                            method='fibonacci',
                            num_samples=1e5,
                            los = True,
                            reflection=True,
                            diffraction=True,
                            check_scene=True,
                            scattering=False,
                            edge_diffraction=True,
                            scat_keep_prob=0.01)
    
    ray_num = len(paths.types.numpy()[0])
    if ray_num == 0:
        building_pixel.append(0)
    else:
        building_pixel.append(255)

    print(f"ray number : {ray_num}")

    # UE * path * properties
    ray_properties = [] #gain, phase, delay, AOD_hor, AOD_ver, AOA_hor, AOA_ver

    for r in range(ray_num):
        path_idx = r
        path_coefficients = paths.a[0,0,0,0,0,path_idx, 0].numpy()
        if path_coefficients == 0: # Skip paths with zero gain
            continue

        path_gain = np.abs(path_coefficients)
        path_phase = np.angle(path_coefficients)
        path_delay = paths.tau[0,0,0,path_idx].numpy()
        path_AOD_ver = paths.theta_t[0,0,0,path_idx]
        path_AOD_hor = paths.phi_t[0,0,0,path_idx]
        path_AOA_ver = paths.theta_r[0,0,0,path_idx]
        path_AOA_hor = paths.phi_r[0,0,0,path_idx]

        ray_properties.append([path_gain, path_phase, path_delay, path_AOD_hor, path_AOD_ver, path_AOA_hor, path_AOA_ver])

    while len(ray_properties) < 297:
        ray_properties.append([0, 0, 0, 0, 0, 0, 0])
    
    ray_properties = np.array(ray_properties)
    np.save(f"NSYSU_grid/data/ue_data_{ue}.npy", ray_properties)


    if np.all(ray_properties[:][0] == 0):
        zero_gain_ue.append(ue)
    
    print(f"UE: {ue}/{batch_size} done")
    scene.remove(f"rx-{ue}")


zero_gain_ue = np.array(zero_gain_ue)
np.save(f"NSYSU_grid/zero_gain_ue.npy", zero_gain_ue)

building_pixel = np.array(building_pixel)
np.save(f"NSYSU_grid/building_pixel.npy", building_pixel)




ray number : 15
UE: 0/13000 done
ray number : 27
UE: 1/13000 done
ray number : 3
UE: 2/13000 done
ray number : 18
UE: 3/13000 done
ray number : 9
UE: 4/13000 done
ray number : 13
UE: 5/13000 done
ray number : 10
UE: 6/13000 done
ray number : 4
UE: 7/13000 done
ray number : 6
UE: 8/13000 done
ray number : 17
UE: 9/13000 done
ray number : 13
UE: 10/13000 done
ray number : 14
UE: 11/13000 done
ray number : 15
UE: 12/13000 done
ray number : 18
UE: 13/13000 done
ray number : 17
UE: 14/13000 done
ray number : 2
UE: 15/13000 done
ray number : 15
UE: 16/13000 done
ray number : 19
UE: 17/13000 done
ray number : 9
UE: 18/13000 done
ray number : 10
UE: 19/13000 done
ray number : 29
UE: 20/13000 done
ray number : 1
UE: 21/13000 done
ray number : 17
UE: 22/13000 done
ray number : 9
UE: 23/13000 done
ray number : 4
UE: 24/13000 done
ray number : 19
UE: 25/13000 done
ray number : 17
UE: 26/13000 done
ray number : 10
UE: 27/13000 done
ray number : 19
UE: 28/13000 done
ray number : 11
UE: 29/13000 done